# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [4]:
# Write your code below.

%reload_ext dotenv
%dotenv 

In [5]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [7]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv('PRICE_DATA')
parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet"))
dd_px = dd.read_parquet(PRICE_DATA).set_index("ticker")

print(dd_px.columns)


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sector',
       'subsector', 'year'],
      dtype='object')


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [8]:
# Write your code below.
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag = x['Close'].shift(1), Adj_close_lag = x['Adj Close'].shift(1))
).assign(
    returns = lambda x: x['Adj Close']/x['Adj_close_lag'] - 1
).assign(
    hi_lo_range = lambda x: (x['High']-x['Low'])
))


C:\Users\kiril\AppData\Local\Temp\ipykernel_15392\226744910.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [9]:
# Write your code below.

pd_feat = dd_feat.compute()

c:\Users\kiril\miniconda3\Lib\site-packages\pandas\core\frame.py:717: DeprecationWarning: Passing a BlockManager to DataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(


In [10]:
pd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag,Adj_close_lag,returns,hi_lo_range
ticker,,,,,,,,,,,,,,
OXY,2016-01-04,66.975945,67.764603,66.257172,67.295403,51.971539,4785736,Energy,Oil & Gas Exploration & Production,2016,NaN,NaN,NaN,1.507431
OXY,2016-01-05,67.065796,68.593193,66.666473,68.413498,52.835033,4475609,Energy,Oil & Gas Exploration & Production,2016,67.295403,51.971539,0.016615,1.926720
OXY,2016-01-06,66.636528,66.995911,64.649910,65.328751,50.452713,6751978,Energy,Oil & Gas Exploration & Production,2016,68.413498,52.835033,-0.045090,2.346001
OXY,2016-01-07,63.961082,65.568344,63.581726,64.150757,49.542961,6573775,Energy,Oil & Gas Exploration & Production,2016,65.328751,50.452713,-0.018032,1.986618
OXY,2016-01-08,64.649910,64.839584,62.833000,63.062611,48.702602,6213563,Energy,Oil & Gas Exploration & Production,2016,64.150757,49.542961,-0.016962,2.006584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INTU,2009-12-24,30.740000,30.990000,30.590000,30.940001,27.573114,727800,Information Technology,Application Software,2009,30.740000,27.394882,0.006506,0.400000
INTU,2009-12-28,30.879999,31.049999,30.780001,31.000000,27.626589,1453700,Information Technology,Application Software,2009,30.940001,27.573114,0.001939,0.269999
INTU,2009-12-29,30.959999,31.040001,30.770000,30.959999,27.590939,1265100,Information Technology,Application Software,2009,31.000000,27.626589,-0.001290,0.270000


In [11]:
pd_feat_window = pd_feat["returns"].rolling(10).mean()
pd_feat_window

ticker
OXY          NaN
OXY          NaN
OXY          NaN
OXY          NaN
OXY          NaN
          ...   
INTU    0.003027
INTU    0.004120
INTU    0.001874
INTU    0.002756
INTU    0.001661
Name: returns, Length: 2746333, dtype: float64

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

Since Dask dataframes also has rolling().mean function it is not necessary

+ Would it have been better to do it in Dask? Why?

Whether Dask is "better" depends on your specific use case. If you're dealing with large datasets that exceed memory capacity, require parallel processing, or need to scale out to multiple machines, Dask offers significant advantages. It allows you to handle big data scenarios that Pandas alone cannot manage efficiently.

However, for smaller datasets or tasks that are not compute-intensive, sticking with Pandas might be simpler and perfectly adequate. Always consider factors like dataset size, available computational resources, and the need for parallelism when deciding whether to use Dask or not.

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.